In [1]:
%load_ext autoreload
%autoreload 2
import time, base64,ssl, os, re,json5, pprint,random , math, hashlib, inspect,requests
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
from app.operation_executor import OperationExecutor
import pandas as pd
import numpy as np
from io import StringIO
import json

In [ ]:
from playwright.sync_api import sync_playwright

with sync_playwright() as p:
    browser = p.chromium.launch(headless=False)  # headless=True runs invisibly
    page = browser.new_page()

    page.goto("https://www.google.com")
    page.fill("input[name='q']", "borgir")
    page.press("input[name='q']", "Enter")

    page.wait_for_timeout(3000)  # 3 seconds to admire borgir results
    browser.close()


In [4]:
import json
import re
import pandas as pd
from bs4 import BeautifulSoup

def parse_html_table(html):
    """Parse one HTML table into DataFrame with Date, Subject, Remarks, Link."""
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table")
    if not table:
        return None

    rows = []
    for tr in table.select("tbody tr"):
        cols = tr.find_all("td")
        if len(cols) < 3:
            print("There are less than three cols, skipping")
            continue

        date = cols[0].get_text(strip=True)
        remarks = cols[2].get_text(strip=True)

        a = cols[1].find("a")
        subject = a.get_text(" ", strip=True) if a else cols[1].get_text(strip=True)
        link = ""
        if a and a.get("onclick"):
            onclick = a["onclick"]
            if "newwindow1" in onclick:
                start = onclick.find("'") + 1
                end = onclick.rfind("'")
                link = onclick[start:end]

        rows.append([date, subject, remarks, link])

    return pd.DataFrame(rows, columns=["Date", "Subject", "Remarks", "Link"])

def clean_sheet_name(name, fallback="Sheet"):
    safe = re.sub(r'[\[\]\:\*\?\/\\]', "_", str(name))
    return safe[:31] if safe else fallback

def cache_to_excel(cache_file, excel_out="Orders_All.xlsx"):
    with open(cache_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    used_names = {}  # track counts per sheet name

    with pd.ExcelWriter(excel_out, engine="xlsxwriter") as writer:
        for scraped in data["records"][0]["scraped_data"]:
            if not scraped.get("data_present"):
                continue

            action = scraped.get("action", "unknown")
            webpage = scraped.get("webpage", "")
            for resp in scraped.get("response", []):
                if resp.get("type") != "table_html":
                    continue

                df = parse_html_table(resp["value"])
                if df is None or df.empty:
                    continue

                titles = resp.get("title", [])
                base_name = clean_sheet_name(titles[-1] if titles else action)

                count = used_names.get(base_name, 0) + 1
                used_names[base_name] = count
                sheet_name = base_name if count == 1 else clean_sheet_name(f"{base_name}_{count}")


                meta_rows = [["Action", action], ["Webpage", webpage]]
                for i, t in enumerate(titles, start=1):
                    meta_rows.append([f"Header{i}", t])
                meta = pd.DataFrame(meta_rows, columns=["Meta", "Value"])
                meta.to_excel(writer, sheet_name=sheet_name, index=False, startrow=0)

                df_visible = df[["Date", "Subject", "Remarks","Link"]]
                startrow = len(meta) + 2
                df_visible.to_excel(writer, sheet_name=sheet_name, index=False, startrow=startrow)

                # ws = writer.sheets[sheet_name]
                # for i, (txt, url) in enumerate(zip(df["Subject"], df["Link"]), start=startrow+1):
                #     if url:
                #         ws.write_url(i, 1, url, string=txt)

    print(f"Excel saved: {excel_out}")


# Example usage
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\2025-09-23\25-09-23T15-32-09_cache.json"
cache_to_excel(path, "Orders_example_All.xlsx")


There are less than three cols, skipping
Excel saved: Orders_example_All.xlsx


In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
path = r"C:\Users\rando\Office Projects\outputs\scrape_output\cache\2025-09-23\25-09-23T22-31-06_cache.json"
data = Helper.load_json(path)
OperationExecutor.generate_cache_doc_report(data)

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://punjabandsind.bank.in/module/bulk-deposit-rates")

# find the anchor or button linking to PDF
link = driver.find_element(By.PARTIAL_LINK_TEXT, ".pdf")
pdf_url = link.get_attribute("href")

print("PDF URL:", pdf_url)


PDF URL: https://punjabandsind.bank.in/system/uploads/bulk_deposit_rate/2025100413032313708.pdf


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://punjabandsind.bank.in/module/bulk-deposit-rates")

# find the anchor or button linking to PDF
link = driver.find_element(By.PARTIAL_LINK_TEXT, ".pdf")
pdf_url = link.get_attribute("href")

print("PDF URL:", pdf_url)


PDF URL: https://punjabandsind.bank.in/system/uploads/bulk_deposit_rate/2025100413032313708.pdf


In [3]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
from urllib.parse import urljoin
import os
ops = OperationExecutor()
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\2025-09-26\25-09-26T15-44-07_cache.json"
# path = r"25-09-08T18-33-09_cache.json5"
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\2025-09-26\25-09-26T15-39-53_cache.json"

    
root_file = os.path.basename(path).split("_")[0]
function_to_execute = {
"primary":[["normalize_df","value","norm_table","table_html"],["sha1","value","SHA_ONE","pdf"]],
"secondary":[["sha1","norm_table","SHA_ONE"]],
}
try:
    data = Helper.load_json(path, typ="json5")
    processed_data = ops.runner(data,function_to_execute)
    Helper.save_json(processed_data,f"{root_file}_process.json")
except Exception as e:
    print(f"\nError in Processing.. Skipping, {e}")


>>Processing Bank of Baroda


In [4]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()

v1 = r"25-09-26T15-44-07_process.json"
v2 = r"25-09-26T15-39-53_process.json"
yesterday = Helper.load_json(v2)
today = Helper.load_json(v1)

result = ops.process_comparison(yesterday,today,key="SHA_ONE")
Helper.save_json(result, "compare-080925.json")

ops.generate_sorted_excel_report(result,output_path="Despoit_Rate_Report.xlsx")

'Despoit_Rate_Report.xlsx'

In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()

path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\cache\1109_1009_compare\compare-080925.json"
result = Helper.load_json(path)
ops.generate_sorted_excel_report(result,output_path="Despoit_Rate_Report.xlsx")

'Despoit_Rate_Report.xlsx'